<a href="https://colab.research.google.com/github/loktissimo/Raketa/blob/main/Raketa_PROD_v5_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title Заполни дату
# @markdown Дата начала выписки

from datetime import datetime, timezone
import IPython
import requests

DATEFROM = '2024-10-01'  #@param {type: "date"}
CLIENT = 'Woerwag' #@param ["Petrovax", "Woerwag"]

def string_to_isotime(date_string):
    datetime_object = datetime.fromisoformat(date_string)
    return datetime_object.strftime("%Y-%m-%dT%H:%M:%S.000Z")

def string_to_humanreadable(date_string):
    datetime_object = datetime.fromisoformat(date_string)
    return datetime_object.strftime("%d %h %Y")

DATE_FROM = string_to_isotime(DATEFROM)
DATE_NOW = datetime.now().strftime("%Y-%m-%dT%H:%M:%S.000Z")
DATE_FILE = datetime.now().strftime("%H:%M-%d.%m.%Y")
DATE_FROM_HUMAN = string_to_humanreadable(DATEFROM)
# print(f"{DATE_FROM = }\n{DATE_NOW = }\n{DATE_FILE = }\n{DATE_FROM_HUMAN}")



''' GLOBAL Variables '''
token = 'XXXXXXXXXXXXXXXXXXXXXXXXXXXXXX'
headers = {'Authorization': f'Bearer {token}', 'accept': 'application/json'}



def get_json(headers):
    '''
    Сбор данных из ракеты по 100 "строк" за проход
    '''

    date_from = DATE_FROM

    ''' Отображение числа записей '''
    rocket_item_counter = 0
    print_rocket_counter = display(IPython.display.Pretty(f'⏳ Получено {rocket_item_counter} записей Ракеты с {DATE_FROM_HUMAN}'), display_id=True)


    services_endpoint = 'https://agency.raketa.travel/v2/services?fromLastUpdate=' + date_from
    json_data_services = requests.get(services_endpoint, headers=headers).json()
    json_last_date = json_data_services[len(json_data_services)-1]['lastUpdate']


    while json_last_date[:10] != DATE_NOW[:10]:
        date_from = string_to_isotime(json_last_date)
        url = "https://agency.raketa.travel/v2/services?fromLastUpdate=" + date_from
        new_request = requests.get(url, headers=headers).json()

        if not new_request: break

        rocket_item_counter += len(new_request)
        print_rocket_counter.update(IPython.display.Pretty(f'⏳ Получено {rocket_item_counter} записей Ракеты с {DATE_FROM_HUMAN}'))

        # print(f'LastUpdate field: {json_data_services[len(json_data_services)-1]["lastUpdate"]}')
        # print(f'{date_from = }')
        # print(f'{new_request = }')

        json_data_services.extend(new_request)
        json_last_date = new_request[len(new_request)-1]['lastUpdate']

    return json_data_services


''' Запрос по API массива данных из Ракеты '''
json_data_services = get_json(headers)


print(f"\n✔️ Готово. Клиент - {CLIENT}")



⏳ Получено 1268 записей Ракеты с 01 Oct 2024


✔️ Готово. Клиент - Woerwag


In [ ]:
#  @title Загрузи .xlsx файл

import pandas as pd
import io

from google.colab import files
uploaded = files.upload()

filename = list(uploaded.keys())

# df = pd.read_excel(io.BytesIO(uploaded[filename[0]]), header=[1])

# Проверяем файл, что он подходит для работы далее
# headers = ['Имя позиции в счете',
#         'Путешественник',
#         'Расчетный код перевозчика',
#         '№ продукта',
#         'Дата выписки',
#         'Табельный номер',
#         'ЦФО',
#         'код ЦФО',
#         'Дата начала оказания услуги',
#         'Дата окончания оказания услуги',
#         'Грейд',
#         'Город прибытия',
#         'Тип продукта',
#         'Сумма такс',
#         'Сумма НДС в счете',
#         'Ставка НДС в счете',
#         'Сумма счета',
#         'ИНН поставщика',
#         'Клиент (полное название)',
#         'Статус продукта',
#         '№ заказа',
#         'Наименование ИНН поставщика',
#         'Комментарии',
#         '№ реализации',
#         'Тариф',
#         'Выписывающий агент']

# if list(df.columns) != headers:
#     print("\n\n😔 Неверный исходный файл! Загрузи другой.")


print(f"\n✔️ Готово. Файл - {filename[0]}")


Saving Верваг НДС в таксах_01-15.11.2024_доп.xlsx to Верваг НДС в таксах_01-15.11.2024_доп.xlsx

✔️ Готово. Файл - Верваг НДС в таксах_01-15.11.2024_доп.xlsx


In [ ]:
# @title Запускай анализ файла

import requests
import pandas as pd
import re
import io
from time import sleep
from ipywidgets import IntProgress
from IPython.display import display
# import IPython
# from tables import file


def ticket_to_str(code):
    '''
    Получение номеров из pandas в виде str
    '''
    try:
        return str(int(code))    #str(int) потому что pandas может поменять тип на float.
        # return str(code)
    except:
        print(f'⚠️ С номером билета / кодом АК ({code}) проблема. Передай разработчикам!')
        return None


def validate_ticket(code_ak, ticket_number, service_type):
    '''
    Получение номера билета на основании типа услуги
    '''
    if service_type == "Железнодорожные билеты" or service_type == "Гостиницы" or service_type == "Транспортные услуги":
        return ticket_to_str(ticket_number)
    elif service_type == "Авиабилеты":
        return "".join( (ticket_to_str(code_ak), ticket_to_str(ticket_number)) )
    else:
        return None


def find_order_clientid(ticket_number, json):
    '''
    Get order id, client id by ticket number
    '''
    result = [None, None]

    for element in json:
        if element['type'] == 'avia' \
            and element['avia']['tickets'] \
            and element['avia']['tickets'][0]['number'] == ticket_number:
            result = [element['orderId'], element['avia']['tickets'][0].get('passengerId')]
            return result

        elif element['type'] == 'rail' \
            and element['rail']['forward'][0]['tickets'] \
            and element['rail']['forward'][0]['tickets'][0]['number'] == ticket_number:
            result = [element['orderId'], element['rail']['forward'][0]['tickets'][0].get('passengerId')]
            return result

        elif element['type'] == 'hotel' \
            and element['hotel']['bookingReference'] \
            and element['hotel']['bookingReference'] == ticket_number:
            result = [element['orderId'], element['travellers'][0]]
            return result

        elif element['type'] == 'transfer' \
            and element['transfer']['bookingReference'] \
            and element['transfer']['bookingReference'] == ticket_number:
                result = [element['orderId'], element['travellers'][0]]
                return result

        elif element['type'] == 'misc' and element['files']:    # Поиск имён файлов для offline-билетов
            filenames_list = [item.get('fileName') for item in element['files']]
            filenames_strings = [s for s in filenames_list if ticket_number in s]
            if filenames_strings:
                result = [element.get('orderId'), element.get('customer')]
                return result

    print(f"⚠️ Информация о билете ({ticket_number}) не найдена")
    return result


def find_tabnum_cfo_code_fio(client_id, headers):
    '''
    Найти ЦФО, Код ЦФО, табельный номер + ФИО
    '''
    client_id = str(client_id)
    profiles_endpoint = f'https://agency.raketa.travel/v2/profiles/{client_id}'

    json_data_profiles = requests.get(profiles_endpoint, headers=headers).json()

    fio = " ".join([json_data_profiles['lastName'], json_data_profiles['firstName'], json_data_profiles['middleName']])

    if json_data_profiles.get('extrafields'):
        return [json_data_profiles['code'], json_data_profiles['costCenter'], json_data_profiles['extrafields'][0]['value'], fio]
    else:
        return [json_data_profiles['code'], json_data_profiles['costCenter'], None, fio]


def find_brand(order_id, headers):
    '''
    Поиска Бренда
    '''
    brand_endpoint = f'https://agency.raketa.travel/v2/orders/{order_id}'
    json_data_brand = requests.get(brand_endpoint, headers=headers).json()

    if json_data_brand.get('extrafields'):
        ''' С номером элеманта массива [1] могут быть *фокусы* '''
        return str(json_data_brand['extrafields'][1]['label'])


def find_businesstrip(order_id, client, headers):
    '''
    Поиск Цель командировки
    '''
    if client != 'Woerwag':
        businesstrip_endpoint = f'https://agency.raketa.travel/v2/orders/{order_id}/businessTrip'
        json_data_businesstrip = requests.get(businesstrip_endpoint, headers=headers).json()
        if json_data_businesstrip.get('purpose'):
            return str(json_data_businesstrip['purpose'])
        else:
            print(f'{order_id = }') #debug
            return None

    elif client == 'Woerwag':
        brand_endpoint = f'https://agency.raketa.travel/v2/orders/{order_id}'
        json_data_brand = requests.get(brand_endpoint, headers=headers).json()

        if json_data_brand.get('extrafields'):
            return str(json_data_brand['extrafields'][2]['label'])
        else:
            print(f'{order_id = }') #debug
            return None



def collect_data_and_generate_df(**kwargs):
    '''
    Собираем данные и формуруем нормализованный DataFrame
    '''
    # filename = list(uploaded.keys())      #  !!!!!!!!!!!!!!! удалить, если всё работает.
    print(f'Анализируем файл: {filename[0]}\n')

    ''' Загружаем данные из .xls в Pandas DataFrame '''
    df = pd.read_excel(io.BytesIO(uploaded[filename[0]]), header=[1])

    ''' Статусбар по красоте '''
    bar_max_count = len(df.index) - 2
    bar = IntProgress(min=0, max=bar_max_count) # instantiate the bar
    display(bar) # display the bar


    ''' Добавляем в DataFrame все необходимые колонки в конец таблицы
        Мы приняли подход, когда мы сначала заполняем исходную таблицу всеми полями,
        а затем из нее вырезаем нужную клиентскую форму '''
    df['№ п/п'] = ""
    df['Номер заказа в СОБ'] = ""
    df['№ акта'] = ""
    df['дата акта'] = ""
    df['Цель командировки'] = ""
    df['номер строки билета'] = ""
    df['Бренд'] = ""
    df['Номер билета к обмену'] = ""
    df['Номер клиента'] = ""
    df['Стоимость услуги без НДС'] = ""
    df['Номер СЧФ'] = ""
    df['Номенклатура'] = ""
    # df['Содержание'] = ""     # Удаляем ?
    df['NON/ded'] = ""
    df['Tax code'] = ""
    #print(df.columns)
    #print(df.head())


    ''' Проход по строкам файла .xls '''
    for index, row in df.iterrows():

        ''' Статусбар по красоте '''
        bar.value = index   # signal to increment the progress bar

        # print(f'{row.iloc[2] = } {type(row.iloc[2]) = } \n'
        #     f'{row.iloc[3] = } {type(row.iloc[3]) = } \n'
        #     f'{row.iloc[12] = } {type(row.iloc[12]) = }')

        ticket_number = validate_ticket(row[2], row[3], row[12])
        # ticket_number = validate_ticket(row.iloc[2], row.iloc[3], row.iloc[12]) # ПРОВЕРИТЬ В СЛЕДУЮЩЕМ СПРИНТЕ
        if ticket_number:
            order_id, client_id = find_order_clientid(ticket_number, json_data_services)
            if order_id and client_id:
                tabnum, cfo, cfo_code, fio = find_tabnum_cfo_code_fio(client_id, headers)
                businesstrip = find_businesstrip(order_id, CLIENT, headers)
                brand = find_brand(order_id, headers)

                '''  _debug_ - проверочная информация по найденным данным '''
                if kwargs.get('_debug_', False):
                    print(f"{'№ билета' : <17} {ticket_number}\n"   # не добавляем в DataFrame
                            f"{'№ СОБ (order_id)' : <17} {order_id}\n"
                            f"{'№ Клиента (client_id)' : <17} {client_id}\n"
                            f"{'ФИО' : <17} {fio}\n"
                            f"{'Цель ком.' : <17} {businesstrip}\n"
                            f"{'Бренд' : <17} {brand}\n"
                            f"{'Таб.номер' : <17} {tabnum}\n"
                            f"{'ЦФО' : <17} {cfo}\n"
                            f"{'Код ЦФО' : <17} {cfo_code}\n")


                ''' Add collected data to DataFrame '''
                # df.at[index, 'Номер билета'] = ticket_number  # оставляем существующий из исходного файла
                df.at[index, 'Номер заказа в СОБ'] = order_id
                df.at[index, 'Путешественник'] = fio
                df.at[index, 'Номер клиента'] = client_id
                df.at[index, 'Цель командировки'] = businesstrip
                df.at[index, 'Бренд'] = brand
                df.at[index, 'Табельный номер'] = tabnum
                df.at[index, 'ЦФО'] = cfo
                df.at[index, 'код ЦФО'] = cfo_code

    print(f"\n✔️ Готово")
    return df


''' Записываем DataFrame в переменную '''
df_normalized = collect_data_and_generate_df(_debug_ = False)



Анализируем файл: Верваг НДС в таксах_01-15.11.2024_доп.xlsx



IntProgress(value=0, max=6)

<ipython-input-10-ac9b2cab89fe>:182: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  ticket_number = validate_ticket(row[2], row[3], row[12])
<ipython-input-10-ac9b2cab89fe>:182: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  ticket_number = validate_ticket(row[2], row[3], row[12])



✔️ Готово


In [ ]:
# @title Нажимай для формирования итогового файла

result_filename = f"{CLIENT}_report_{DATE_FILE}.xlsx"

petrovax_rename = {'Имя позиции в счете':'Наименование услуги',
                    '№ продукта':'номер билета',
                    'Дата выписки':'Дата оформления билета',
                    'Дата начала оказания услуги':'Дата начала командировки (вылета)',
                    'Дата окончания оказания услуги':'Дата окончания командировки (прилета)',
                    'Город прибытия':'Место командировки',
                    'Тип продукта':'Тип услуги',
                    'Сумма НДС в счете':'Сумма НДС',
                    'Ставка НДС в счете':'Ставка НДС',
                    'Сумма счета':'Стоимость услуги с НДС',
                    'ИНН поставщика':'ИНН поставщика (авиакомпании, ржд)',
                    'Клиент (полное название)':'ЮЛ',
                    'Статус продукта':'Вид операции',
                    '№ заказа':'Номер заказа в МОМ',
                    'Путешественник':'ФИО пассажира'}

petrovax_reindex = ['№ п/п',
                    'Наименование услуги',
                    'Номер заказа в СОБ',
                    'ФИО пассажира',
                    '№ акта',
                    'дата акта',
                    'номер билета',
                    'Дата оформления билета',
                    'Табельный номер',
                    'ЦФО',
                    'код ЦФО',
                    'Дата начала командировки (вылета)',
                    'Дата окончания командировки (прилета)',
                    'Цель командировки',
                    'Место командировки',
                    'Тип услуги',
                    'Стоимость услуги без НДС',
                    'Сумма НДС',
                    'Ставка НДС',
                    'Стоимость услуги с НДС',
                    'номер строки билета',
                    'ИНН поставщика (авиакомпании, ржд)',
                    'ЮЛ',
                    'Бренд',
                    'Номер билета к обмену',
                    'Вид операции',
                    'Номер заказа в МОМ']

woerwag_rename = {'Имя позиции в счете':'Содержание',
                    'Дата начала оказания услуги':'Даты отправления',
                    'Наименование ИНН поставщика':'Перевозчик',
                    'ЦФО':'Кост-центр путешественника',
                    'Сумма счета':'Сумма',
                    'Ставка НДС в счете':'Ставка НДС',
                    'Сумма НДС в счете':'Сумма НДС',
                    'Стоимость услуги без НДС':'Сумма без НДС',
                    'Цель командировки':'Цель поездки',
                    'Тип продукта':'Вид услуги',
                    'Бренд':'Препарат',
                    '№ продукта':'номер билета',
                    'Номер заказа в СОБ':'номер заказа в СОБ'}

woerwag_reindex = ['Номер СЧФ',
                    'Номенклатура',
                    'Содержание',
                    'Путешественник',
                    'Маршрут',
                    'Даты отправления',
                    'Перевозчик',
                    'Кост-центр путешественника',
                    'Сумма',
                    'Ставка НДС',
                    'Сумма НДС',
                    'Сумма без НДС',
                    'Цель поездки',
                    'Вид услуги',
                    'Препарат',
                    'NON/ded',
                    'Tax code',
                    'Комментарии',
                    'номер билета',
                    'номер заказа в СОБ']


''' Удаляем ненужные колонки. Ненужные колонки можно и не удалять. На этапе выстраивания последовательности колонок
мы перекладываем один df в клиентский с реиндексом, поэтому в исходной форме можно не делать df.drop. '''
# client_df.drop(['Расчетный код перевозчика',


def map_and_save_client_form(client_df, client_name):
    '''
    Сборка клиентских форм и запись DataFrame в клиентские файлы
    '''

    if client_name == 'Petrovax':
        print(f'⌛ Запускаем форматирование {client_name}')

        ''' Переименовываем колонки'''
        client_df.rename(columns = petrovax_rename, inplace = True )

        ''' Выстраиваем колонки в нужной последовательности'''
        petrovax_df = client_df.reindex(columns=petrovax_reindex)

        ''' Модифицируем колонки'''
        # Вычисляем колонку "Стоимость услуги без НДС"
        petrovax_df['Стоимость услуги без НДС'] = petrovax_df['Стоимость услуги с НДС'] - petrovax_df['Сумма НДС']

        ''' Модифицируем колонку "Тип Услуги" '''
        for index1, row in petrovax_df.iterrows():
          if str(row[15]).startswith('Железнодорожные билеты'):
            if row[1].startswith('ЖД билет'):
              petrovax_df.at[index1, 'Тип услуги'] = 'ЖД билет'
            else:
              petrovax_df.at[index1, 'Тип услуги'] = 'Сборы'

          elif str(row[15]).startswith('Авиабилеты'):
            if row[1].startswith('Сервисный сбор'):
              petrovax_df.at[index1, 'Тип услуги'] = 'Сборы'

            elif row[1].startswith('EMD'):
              if row[1].startswith('EMD на Штраф '):
                petrovax_df.at[index1, 'Тип услуги'] = 'Штраф'
              else:
                petrovax_df.at[index1, 'Тип услуги'] = 'Допуслуга авиа'

            else:
              petrovax_df.at[index1, 'Тип услуги'] = 'Авиабилет'

          elif str(row[1]).startswith('Проживание'):
            petrovax_df.at[index1, 'Тип услуги'] = 'Проживание'

          elif str(row[1]).startswith(', '):
            petrovax_df.at[index1, 'Тип услуги'] = 'Трансфер'

          else:
            petrovax_df.at[index1, 'Тип услуги'] = 'Сборы'

        ''' Модифицируем колонку "номер строки билета" '''
        for index_nomer_bileta, row in petrovax_df.iterrows():
          if str(row[1]).startswith('Тариф'):
            petrovax_df.at[index_nomer_bileta, 'номер строки билета'] = '1'
          elif str(row[1]).startswith('Такса ZZ'):
            petrovax_df.at[index_nomer_bileta, 'номер строки билета'] = '3'
          elif str(row[1]).startswith('Таксы и сборы'):
            petrovax_df.at[index_nomer_bileta, 'номер строки билета'] = '2'
          else:
            print(f'Не могу установить номер строки билета для строки {index_nomer_bileta}.')

        ''' Модифицируем колонку "Вид операции" '''
        for index_vid_opercii, row in petrovax_df.iterrows():
          if str(row[15]) in ["Допуслуга авиа", "Штраф", "Проживание", "Трансфер", "Сборы"]:
           petrovax_df.at[index_vid_opercii, 'Вид операции']= 'Сборы'

        #print(petrovax_df.columns)
        petrovax_df.to_excel(result_filename, index=False)
        print(f'\n✔️ Файл {result_filename} записан')



    elif client_name == 'Woerwag':
        print(f'⌛ Запускаем форматирование {client_name}')

        ''' Переименовываем колонки '''
        client_df.rename(columns = woerwag_rename, inplace = True )

        ''' Выстраиваем колонки в нужной последовательности '''
        woerwag_df = client_df.reindex(columns=woerwag_reindex)

        ''' Модифицируем колонки '''

        ''' Модифицируем колонку "Сумма без НДС"'''
        woerwag_df['Сумма без НДС'] = woerwag_df['Сумма'] - woerwag_df['Сумма НДС']

        ''' Модифицирем колонку "Номенклатура" '''
        for index1, row in woerwag_df.iterrows():
          if str(row[13]) == 'Авиабилеты':
            if row[2].startswith('Сервисный сбор'):
              woerwag_df.at[index1, 'Номенклатура'] = 'Сервисный сбор'
            elif row[2].startswith('Такса ZZ'):
              woerwag_df.at[index1, 'Номенклатура'] = 'Такса ZZ'
            elif row[2].startswith('Таксы без НДС'):
              woerwag_df.at[index1, 'Номенклатура'] = 'Таксы без НДС'
            elif row[2].startswith('Таксы и сборы НДС0'):
              woerwag_df.at[index1, 'Номенклатура'] = 'Таксы и сборы НДС0'
            elif row[2].startswith('Таксы и сборы НДС10'):
              woerwag_df.at[index1, 'Номенклатура'] = 'Таксы и сборы НДС10'
            else:
              woerwag_df.at[index1, 'Номенклатура'] = 'Тариф'

          if str(row[13]) == 'Гостиницы':
              woerwag_df.at[index1, 'Номенклатура'] = 'Проживание'

          if str(row[13]) == 'Железнодорожные билеты':
            if row[2].startswith('ЖД билет'):
              woerwag_df.at[index1, 'Номенклатура'] = 'ЖД билет'
            elif row[2].startswith('Сервисный сбор ИМ'):
              woerwag_df.at[index1, 'Номенклатура'] = 'Сервисный сбор ИМ'
            else:
              woerwag_df.at[index1, 'Номенклатура'] = 'Сервисный сбор'

        ''' Модифицирем колонку "Вид услуги" '''
        for index_vid_uslugi, row in woerwag_df.iterrows():
          if str(row[13]) == 'Железнодорожные билеты':
            woerwag_df.at[index_vid_uslugi, 'Вид услуги'] = 'ЖД билет'
          elif str(row[13]) == 'Авиабилеты':
            woerwag_df.at[index_vid_uslugi, 'Вид услуги'] = 'Авиабилет'
          elif str(row[13]) == 'Гостиницы':
            woerwag_df.at[index_vid_uslugi, 'Вид услуги'] = 'Отель'

        ''' Модифицирем колонку "Перевозчик" '''
        for index_perevozchik, row in woerwag_df.iterrows():
          if row[1].startswith('ЖД билет'):
            woerwag_df.at[index_perevozchik, 'Перевозчик'] = 'РЖД'
          elif row[1].startswith('Сервисный сбор ИМ'):
            woerwag_df.at[index_perevozchik, 'Перевозчик'] = 'Инновационная мобильность'
          elif row[1].startswith('Сервисный сбор'):
            woerwag_df.at[index_perevozchik, 'Перевозчик'] = 'ООО "КОРПОРЭЙТ ТРЭВЕЛ"'


        ''' Записываем файл '''
        woerwag_df.to_excel(result_filename, index=False)
        print(f'\n✔️ Файл {result_filename} записан')

    else:
        print(f'❌Ошибка! Клиента {client_name} не существует')

    return None


''' Создаем целевой файл для клиента '''
map_and_save_client_form(df_normalized, CLIENT)


⌛ Запускаем форматирование Woerwag

✔️ Файл Woerwag_report_10:37-02.12.2024.xlsx записан


<ipython-input-11-2402d479ef17>:176: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if str(row[13]) == 'Авиабилеты':
<ipython-input-11-2402d479ef17>:177: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if row[2].startswith('Сервисный сбор'):
<ipython-input-11-2402d479ef17>:179: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  elif row[2].startswith('Такса ZZ'):
<ipython-input-11-2402d479ef17>:181: FutureWarning: Series.__getitem__ treatin